# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

In [2]:
# read in the data 

data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [3]:
# only retain the relevant columns 

data = data[['race', 'call']]
data.head()

,race,call
0,w,0.0
1,w,0.0
2,b,0.0
3,b,0.0
4,w,0.0


In [4]:
# calculate the mean call rate for each race

means = data.groupby('race').agg(np.mean)
print 'mean callback rate for each race:'
means

mean callback rate for each race:


,call
race,
b,0.064476
w,0.096509


In [5]:
odds_ratio = (0.096509 / (1.- 0.096509)) / (0.064476 / (1. - 0.064476))
print 'odds ratio (white to black):', odds_ratio

odds ratio (white to black): 1.54988984103


<div class="alert alert-info">

White sounding names are $55\%$ <strong>more likely</strong> to receive a call compared to black sounding names. The <strong>null hypothesis</strong> is that the difference of means is zero, and the <strong>alternative hypothesis</strong> is that the difference of mean is nonzero. We use the <strong>two sample t-test</strong> to check significance. 

</div>

In [6]:
# calculate the standard deviation by the race

stds = data.groupby('race').agg(np.std)
print 'standard deviations for each race:'
stds

standard deviations for each race:


,call
race,
b,0.245650
w,0.295349


In [7]:
# calculate the total number of candidates for each race 

counts = data.groupby('race').size().astype(float)
print 'total number of samples for each race:'
counts

total number of samples for each race:


race
b    2435.0
w    2435.0
dtype: float64

<div class='alert alert-info'>
For either race, the call column is a <strong>Bernoulli distribution</strong>. CLT applies, and since the samples have large size, we use the <strong>normal distribution</strong>, rather than the t-distribution. 
</div>

In [8]:
# calculate the standard error, the difference of the means, the z-value and the p-value

stderr = np.sqrt(stds.loc['b']**2/counts.loc['b'] + stds.loc['w']**2/counts.loc['w']).values[0]
diff_means = (means.loc['w'] - means.loc['b']).values[0]
print 'difference of means:', diff_means
zvalue = diff_means / stderr
print 'z-score:', zvalue
print 'p-value', st.norm.sf(zvalue)

# 99% confidence interval for the difference between the means 

z_crit = st.norm.isf(0.005)
confidence_interval = (diff_means - z_crit*stderr, diff_means + z_crit*stderr)

print '99% confidence interval for the difference of the means', confidence_interval

difference of means: 0.0320329
z-score: 4.11471
p-value 1.93836972613e-05
99% confidence interval for the difference of the means (0.011980103131064651, 0.052085606579056504)


<div class='alert alert-info'>

The difference between the mean call rate by race is statistically significant at an $\alpha$ value of $0.01$. That is, we <strong>reject the null</strong> hypothesis. The difference of means is nonzero with high statistical significance. 

</div>

<div class='alert alert-info'>
<h3> Concluding Remarks: </h3> 
<p>
The names which sounded white were <strong> $55\%$ more likely to receive a call </strong> for interview compared to the names which sounded black. This does **not** mean that race/name is the most important factor in callback success - Some other factor that we did not analyze, for example, education or the amount of experience, could be more influential. Since the resumes were all identical, our analysis does mean that race/name certainly impacts callback success rate. For a more thorough analysis, other variables such as the amount of experience or education should be controlled for. 
</p>
</div>